# Pré-processamento para interpolação de voxel
* Todos os conjuntos de dados foram pré-processados para obter o tamanho de voxel isométricos de (1x1x1). Isso foi feito usando a biblioteca SimpleITK, pois esta permite mudar o tamanho do voxel em precisar manupular manualmente a affine dos volumes. Essa biblioteca tem algumas funções que faz isso automaticamente, garantindo a compatibilidade dos dados para o treinamento. As imagens e as segmentações passaram por esse pré-processamento. Alguns volumes dos conjuntos de dados do CC359 já tem os voxels isométricos e um desses foi usado como referência para aplicar nos outros. 

In [1]:
from scipy.ndimage import zoom
import SimpleITK as sitk
import nibabel as nib
import subprocess
import numpy as np
import glob
import os

def save_nii_zoom(input_paths: str, ref_path: str):
    '''
    Documentação
    '''
    for input_path in input_paths:
        T1_file = 'brain_T1w.nii.gz'
        sitk_T1 = sitk.ReadImage(os.path.join(input_path, T1_file))
        origin = sitk_T1.GetOrigin()
        spacing = sitk_T1.GetSpacing()
        directions = sitk_T1.GetDirection()

        mask_T1_file = 'fix_brainmask_T1w.nii.gz'
        sitk_mask_T1 = sitk.ReadImage(os.path.join(input_path, mask_T1_file))

        pre_affine = nib.load(os.path.join(input_path, T1_file)).affine
        print(f"Pre-affine {pre_affine}")  

        T1_array = sitk.GetArrayFromImage(sitk_T1)
        mask_T1_array = sitk.GetArrayFromImage(sitk_mask_T1)

        input_spacing = np.array(spacing[::-1])  # 1, 1, 2 -> 2, 1, 1
        target_spacing = np.array(sitk.ReadImage(ref_path).GetSpacing())[::-1]
        print(f"Transforming spacing of {T1_array.shape} from {input_spacing} to {target_spacing}")

        T1_iso_array = zoom(T1_array, input_spacing/target_spacing)
        mask_T1_iso_array = zoom(mask_T1_array, input_spacing/target_spacing, order=0)
        print(f"New shape: {T1_iso_array.shape}")

        iso_T1 = sitk.GetImageFromArray(T1_iso_array)
        iso_T1.SetSpacing(target_spacing[::-1])
        iso_T1.SetOrigin(origin)
        iso_T1.SetDirection(directions)
        iso_mask_T1 = sitk.GetImageFromArray(mask_T1_iso_array)
        iso_mask_T1.SetSpacing(target_spacing[::-1])
        iso_mask_T1.SetOrigin(origin)
        iso_mask_T1.SetDirection(directions)



        T1_out_name = "iso_brain_T1w.nii.gz"
        mask_T1_out_name = "iso_brainmask_T1w.nii.gz"

        sitk.WriteImage(iso_T1, os.path.join(input_path, T1_out_name))
        sitk.WriteImage(iso_mask_T1, os.path.join(input_path, mask_T1_out_name))
        post_affine = nib.load(os.path.join(input_path, T1_out_name)).affine
        print(f"Post-affine {post_affine}")

        print("Done")

    return iso_T1, iso_mask_T1

In [2]:
ref_path = "/home/joany/dataset_train/val/CC0126/brain_T1w.nii.gz"
folder_mri = f"/home/joany/brain_dataset/split_dataset/teste"
input_paths = glob.glob(os.path.join(folder_mri, "*"))

image, label = save_nii_zoom(input_paths, ref_path)

Pre-affine [[ -0.          -0.           0.99990082 -98.19879913]
 [ -1.          -0.          -0.         158.90475464]
 [  0.          -1.           0.         107.72076416]
 [  0.           0.           0.           1.        ]]
Transforming spacing of (192, 256, 256) from [0.99990082 1.         1.        ] to [1. 1. 1.]
New shape: (192, 256, 256)
Post-affine [[ -0.           0.           1.         -98.19879913]
 [ -1.          -0.          -0.         158.90475464]
 [  0.          -1.           0.         107.72076416]
 [  0.           0.           0.           1.        ]]
Done
Pre-affine [[ -0.          -0.           0.99990082 -89.52082825]
 [ -1.          -0.          -0.         153.32606506]
 [  0.          -1.           0.          94.70381165]
 [  0.           0.           0.           1.        ]]
Transforming spacing of (192, 256, 256) from [0.99990082 1.         1.        ] to [1. 1. 1.]
New shape: (192, 256, 256)
Post-affine [[ -0.           0.           1.         -89